In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/facebookresearch/mae.git

Cloning into 'mae'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), 829.52 KiB | 9.02 MiB/s, done.


In [ ]:
%cd mae

/content/mae


In [ ]:
!pip install torch
!pip install torchvision
!pip install timm==0.3.2
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 22.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 129.3 MB/s eta 0:00:00


In [ ]:
#To be added to timm/models/layers/helpers.py
#From: https://github.com/huggingface/pytorch-image-models/issues/420#issuecomment-776459842

lines = ["\n", "TORCH_MAJOR = int(torch.__version__.split('.')[0]) \n",
             "TORCH_MINOR = int(torch.__version__.split('.')[1]) \n",
             "if TORCH_MAJOR == 1 and TORCH_MINOR < 8: \n",
             "    from torch._six import container_abcs \n",
             "else: \n",
             "    import collections.abc as container_abcs \n"]

with open('../../usr/local/lib/python3.10/dist-packages/timm/models/layers/helpers.py', 'r') as f:
    code_lines = f.readlines()

with open('../../usr/local/lib/python3.10/dist-packages/timm/models/layers/helpers.py', 'w') as f:
    #res = code_lines[:5] + lines + code_lines[6:]
    code_lines[5] = "import collections.abc as container_abcs"
    f.writelines(code_lines)

In [ ]:
#Change "from torch._six import inf" for "from torch import inf" in util.misc.py

with open('/content/mae/util/misc.py', 'r') as f:
    code_lines = f.readlines()

with open('/content/mae/util/misc.py', 'w') as f:
    code_lines[20] = "from torch import inf"
    f.writelines(code_lines)

In [ ]:
!git clone --branch mae https://github.com/ShulzhenkoPetr/Modal_Challenge.git

Cloning into 'Modal_Challenge'...
remote: Enumerating objects: 372, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 372 (delta 76), reused 75 (delta 35), pack-reused 255
Receiving objects: 100% (372/372), 12.87 MiB | 18.35 MiB/s, done.
Resolving deltas: 100% (209/209), done.


In [ ]:
with open('/content/mae/Modal_Challenge/main_finetune.py', 'r') as f:
    code_lines = f.readlines()

with open('/content/mae/main_finetune.py', 'w') as f:
    f.writelines(code_lines)

In [ ]:
with open('/content/mae/Modal_Challenge/engine_finetune.py', 'r') as f:
    code_lines = f.readlines()

with open('/content/mae/engine_finetune.py', 'w') as f:
    f.writelines(code_lines)

In [ ]:
import torch
import models_vit

checkpoint = torch.load('/content/gdrive/MyDrive/Colab_Notebooks/Modal/mae_pretrain_vit_base.pth')
checkpoint_model = checkpoint['model']

model = models_vit.__dict__['vit_base_patch16'](
        num_classes=48
    )

state_dict = model.state_dict()
for k in ['head.weight', 'head.bias']:
    if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
        print(f"Removing key {k} from pretrained checkpoint")
        del checkpoint_model[k]

model.load_state_dict(checkpoint_model, strict=False)

for param in model.parameters():
    if param.dtype != torch.float32:
        print(param.dtype)


In [ ]:
%run main_finetune.py \
    --model vit_base_patch16 \
    --batch_size 32 \
    --epochs 100 \
    --accum_iter 3 \
    --warmup_epochs 10 \
    --blr 5e-4 \
    --layer_decay 0.65 \
    --weight_decay 0.05 --drop_path 0.1 --reprob 0.25 --mixup 0.8 --cutmix 1.0 \
    --data_path_train /content/gdrive/MyDrive/Colab_Notebooks/Modal/train_train.txt \
    --data_path_val /content/gdrive/MyDrive/Colab_Notebooks/Modal/train_val.txt \
    --nb_classes 48 \
    --cls_token \
    --log_dir /content/gdrive/MyDrive/Colab_Notebooks/Modal/logs \
    --output_dir /content/gdrive/MyDrive/Colab_Notebooks/Modal/checkpoints \
    --hugging_mae

Not using distributed mode
[09:03:19.383815] job dir: /content/mae
[09:03:19.384525] Namespace(batch_size=32,
epochs=100,
accum_iter=3,
model='vit_base_patch16',
input_size=224,
drop_path=0.1,
clip_grad=None,
weight_decay=0.05,
lr=None,
blr=0.0005,
layer_decay=0.65,
min_lr=1e-06,
warmup_epochs=10,
color_jitter=None,
aa='rand-m9-mstd0.5-inc1',
smoothing=0.1,
reprob=0.25,
remode='pixel',
recount=1,
resplit=False,
mixup=0.8,
cutmix=1.0,
cutmix_minmax=None,
mixup_prob=1.0,
mixup_switch_prob=0.5,
mixup_mode='batch',
finetune='',
global_pool=False,
data_path_train='/content/gdrive/MyDrive/Colab_Notebooks/Modal/train_train.txt',
data_path_val='/content/gdrive/MyDrive/Colab_Notebooks/Modal/train_val.txt',
nb_classes=48,
output_dir='/content/gdrive/MyDrive/Colab_Notebooks/Modal/checkpoints',
log_dir='/content/gdrive/MyDrive/Colab_Notebooks/Modal/logs',
device='cuda',
seed=0,
resume='',
start_epoch=0,
eval=False,
dist_eval=False,
num_workers=10,
pin_mem=True,
world_size=1,
local_rank=-1,
dist_on

[09:03:21.341190] 576
[09:03:21.914992] 144


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[09:03:22.670863] Mixup is activated!


[09:03:25.644373] Model = ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (de

...

[09:30:47.544305] Epoch: [99]  [ 0/18]  eta: 0:00:35  lr: 0.000001  loss: 1.1096 (1.1096)  time: 1.9883  data: 1.5897  max mem: 4877


Training Epoch 99: 18it [00:09,  1.98it/s]


[09:30:54.544624] Epoch: [99]  [17/18]  eta: 0:00:00  lr: 0.000001  loss: 1.5943 (1.5207)  time: 0.4986  data: 0.0887  max mem: 4877
[09:30:54.664040] Epoch: [99] Total time: 0:00:09 (0.5061 s / it)
[09:30:54.666993] Averaged stats: lr: 0.000001  loss: 1.5943 (1.5207)


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

[09:30:59.114564] Test:  [0/5]  eta: 0:00:05  loss: 2.5914 (2.5914)  acc1: 40.6250 (40.6250)  acc5: 65.6250 (65.6250)  time: 1.0453  data: 0.8518  max mem: 4877
[09:30:59.628197] Test:  [4/5]  eta: 0:00:00  loss: 2.6888 (2.9031)  acc1: 31.2500 (32.6389)  acc5: 62.5000 (59.7222)  time: 0.3114  data: 0.1705  max mem: 4877
[09:30:59.721223] Test: Total time: 0:00:01 (0.3306 s / it)
[09:30:59.721366] * Acc@1 32.639 Acc@5 59.722 loss 2.903
[09:30:59.722539] Accuracy of the network on the 144 test images: 32.6%
[09:30:59.722600] Max accuracy: 38.89%
[09:30:59.727665] Training time 0:27:34
